## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [44]:
import glob
import os
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)



In [45]:
fs = 125
window_length_s = 8
window_shift_s = 2
overlap = 6
window_length = window_length_s * fs
window_shift = window_shift_s * fs
low_BPM = 40
low_Hz = low_BPM/60 
high_BPM = 240
high_Hz = high_BPM/60 

In [46]:
def LoadTroikaReferanceFile(ref_fl):
    """
    Loads and extracts signals from a troika referance data file.

    Args:
        ref_fls: (str) filepath to a troika .mat file.

    Returns:
        numpy array for reference values.
    """
    ref_data = sp.io.loadmat(ref_fl)['BPM0']
    return ref_data

In [47]:
def BandpassFilter(signal, low_Hz, high_Hz):
    """
    Bandpass filter the signal between 40 & 240 BPM.
    
    Args:
        signal: Numpy array to filter
        fs: sample rate
        pass band:-
        low_Hz: low frequency bound
        high_Hz: high frequency bound
        
        
    Returns:
        Numpy array to filtered signal
    """ 
    b, a = sp.signal.butter(3, (low_Hz, high_Hz), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

In [48]:
def Featurize(ppg, accx, accy, accz):
    """featurization of the accelerometer signal.
    
    Args:
        signal: input signal             
        fs: sampling rate       
        
    Returns:
        n-tuple of accelerometer features
    """
    acc=np.sqrt(accx**2 + accy**2 + accz**2)
    fft_acc=np.fft.rfft(acc,len(acc))
    freqs_acc=np.fft.rfftfreq(len(acc),1/fs)
    fft_ppg=np.fft.rfft(ppg,len(ppg))
    freqs_ppg=np.fft.rfftfreq(len(ppg),1/fs)
    #bandpass 
    fft_ppg[freqs_ppg <= 40/60] = 0.0
    fft_ppg[freqs_ppg >= 240/60] = 0.0
     
    acc_mean = np.mean(acc)
    acc_energy = np.sum(np.square(acc - acc_mean))
    ppg_mean = np.mean(ppg)
    ppg_energy = np.sum(np.square(ppg - ppg_mean))
  
    
    return freqs_ppg,freqs_acc, fft_acc,fft_ppg, acc_energy ,ppg_energy


In [122]:
def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_data = LoadTroikaReferanceFile(ref_fl)
    
    # signals bandpass filter 
    ppg  = BandpassFilter(ppg, low_Hz, high_Hz)
    accx = BandpassFilter(accx, low_Hz, high_Hz)
    accy = BandpassFilter(accy, low_Hz, high_Hz)
    accz = BandpassFilter(accz, low_Hz, high_Hz)

    
    # Compute pulse rate estimates and estimation confidence.
    
    estimate = []
    confidence = []
   
    for i in range(0,len(ppg)- window_length,window_shift):
        ppg_w=ppg[i:i+window_length]
        accx_w=accx[i:i+window_length]
        accy_w=accy[i:i+window_length]
        accz_w=accz[i:i+window_length]
        freqs_ppg,ppg_energy,freqs_acc,acc_energy,fft_ppg,fft_acc = Featurize(ppg_w, accx_w, accy_w, accz_w)
      
        ppg_peak = freqs_ppg[np.argmax(ppg_energy)]
        acc_peak = freqs_acc[np.argmax(acc_energy)]
           
        A = 1        
        while np.abs(ppg_peak - acc_peak) <= 0 and A <=2:
            A += 1
            ppg_peak = freqs_ppg[np.argsort(fft_ppg, axis = 0)[-A]]
            acc_peak = freqs_acc[np.argsort(fft_acc, axis = 0)[-A]]
            
            

        #Estimated freq
        estimate.append(ppg_peak *60)
        
        # compute confidence
        co = np.sum(fft_ppg[(freqs_ppg >= (ppg_peak - low_Hz)) & (freqs_ppg <= (ppg_peak + low_Hz))])
        conf = co/np.sum(fft_ppg)
    
        confidence.append(conf) 
        
        # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
        errors = np.abs(np.diag(estimate - ref_data))
        return errors,confidence

In [123]:
Evaluate()

IndexError: invalid index to scalar variable.

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 